In [1]:
import os
os.chdir("../")
import numpy as np
import pandas as pd
import geopandas as gpd
import pydeck as pdk

from shapely.geometry import Point

In [2]:
mv_path = os.getcwd() + "/data/Moving_Violations_Issued_in_June_2023.csv"
mv_df = pd.read_csv(mv_path)
mv_df.columns = [col.lower() for col in mv_df.columns]
mv_df = mv_df[mv_df.latitude.isna() == False].reset_index(drop=True)

/var/folders/kn/jw0y9v615mjgg3_51g3s73zh0000gn/T/ipykernel_7805/92629993.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  mv_df = pd.read_csv(mv_path)


In [3]:
mv_df["geometry"] = [Point(x,y) for x,y in zip(mv_df.longitude, mv_df.latitude)]
mv_df["location"] = [i.lower().strip() for i in mv_df["location"]]
mv_gdf = gpd.GeoDataFrame(mv_df)

In [4]:
select_columns = ['longitude', 'latitude', 'fine_amount']
pdk_df = (mv_gdf[select_columns].groupby(by=['longitude', 'latitude'])[[
    "fine_amount"]].sum().reset_index())
loc_df = mv_gdf[['longitude', 'latitude', "location"]].drop_duplicates().reset_index(drop=True)
pdk_df = pdk_df.merge(loc_df, how="left", on=["longitude", "latitude"])
pdk_df = pdk_df.iloc[pdk_df.iloc[:, :3].drop_duplicates().index].reset_index(drop=True)
pdk_df["fine_amount_log"] = np.log(pdk_df["fine_amount"]+1)
pdk_df["location"] = pdk_df["location"].str.replace("*", "")

In [5]:
view = pdk.data_utils.compute_view(pdk_df[["longitude", "latitude"]])
view.pitch = 60
view.zoom = 12
view.bearing = 60

column_layer = pdk.Layer(
    "ColumnLayer",
    data=pdk_df,
    get_position=["longitude", "latitude"],
    get_elevation="fine_amount_log",
    elevation_scale=100,
    radius=25,
    pickable=True,
    get_fill_color=[
        "fine_amount * 10", "fine_amount", "fine_amount * 10", 140
    ],
    auto_highlight=True,
)

tooltip = {
    "html": "<b>{location}</b> has fined ${fine_amount} in June 2023.",
    "style": {
        "background": "white",
        "color": "black",
        "font-family": '"Helvetica Neue", Arial',
        "z-index": "10000"
    },
}

r = pdk.Deck(column_layer, initial_view_state=view, tooltip=tooltip)
r.to_html(os.getcwd() + "/output/moving_violations_202306.html")